In [20]:
from IPython.display import display
import IPython
import pandas as pd
from pandas import Series
import numpy as np
from random import randrange
from matplotlib import pyplot
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
import math
from typing import Any, List, Dict, AnyStr, Optional
from pathlib import Path
from glob import glob
from utils.dataset import MarketData, DataPreprocessor, Plotting
from utils.windowgenerator import WindowGenerator, compile_and_fit
import tensorflow as tf
import talib
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
eua_options_cols = ['Date', 'Aggregate Put Open Interest  (R1)', 'Aggregate Call Open Interest  (R1)', 
                    'Aggregate Open Interest  (L1)', 'OPTION OI%', 'PUT/CALL OI']
eua_options = pd.read_excel('../data/data_sheet_latest.xlsx', sheet_name='EUA option-G363')
eua_options['Date'] = pd.to_datetime(pd.to_datetime(eua_options['Date']).dt.date)
eua_options = eua_options[eua_options_cols][eua_options['Date'].dt.year >= 2018].dropna()

In [69]:
display(eua_options.head())

,Date,Aggregate Put Open Interest (R1),Aggregate Call Open Interest (R1),Aggregate Open Interest (L1),OPTION OI%,PUT/CALL OI
1,2024-10-31,206996.0,321757.0,763691.0,0.409111,1.554412
2,2024-10-30,204588.0,322482.0,759379.0,0.409709,1.576251
3,2024-10-29,204538.0,320432.0,760706.0,0.408322,1.566614
4,2024-10-28,204623.0,321162.0,785605.0,0.400937,1.569530
5,2024-10-25,204723.0,321187.0,786404.0,0.400750,1.568886


In [70]:
display(eua_options.tail())

,Date,Aggregate Put Open Interest (R1),Aggregate Call Open Interest (R1),Aggregate Open Interest (L1),OPTION OI%,PUT/CALL OI
1086,2020-08-18,5150.0,28350.0,954401.0,0.033910,5.504854
1087,2020-08-17,5150.0,28350.0,950182.0,0.034056,5.504854
1088,2020-08-14,5150.0,28350.0,946283.0,0.034191,5.504854
1089,2020-08-13,5150.0,28350.0,945581.0,0.034216,5.504854
1090,2020-08-12,5150.0,28350.0,944604.0,0.034250,5.504854


In [151]:
pachis_delta = pd.read_excel('../data/data_sheet_latest.xlsx', sheet_name='25Delta')
dec_cols = ["Date", "Hist Vol", "50D-Hist Vol", "50D", "25D Spread", "butterfly"]
pachis_delta_dec = pachis_delta[dec_cols]
pachis_delta_dec.columns = ["Date", "Hist Vol - 1Y", 
                        "iVol/Hist Vol Spread - Dec", 
                        "50 Delta iVol - Dec", 
                        "25Δ Risk Reversal (Call - Put) - Dec", 
                        "Butterfly - Dec"]
pachis_delta_dec['Date'] = pd.to_datetime(pachis_delta_dec['Date'])
pachis_delta_dec = pachis_delta_dec[pachis_delta_dec['Date'].dt.year > 2018] 
for col in ['Hist Vol - 1Y', 'iVol/Hist Vol Spread - Dec', '50 Delta iVol - Dec']:
    pachis_delta_dec.iloc[:, pachis_delta_dec.columns.get_loc(col)] = pachis_delta_dec[col].replace(' ', np.nan).astype(float)
pachis_delta_dec.iloc[:, pachis_delta_dec.columns.get_loc('iVol/Hist Vol Spread - Dec')] = pachis_delta_dec['iVol/Hist Vol Spread - Dec'].astype(float)
pachis_delta_dec.iloc[:, pachis_delta_dec.columns.get_loc('50 Delta iVol - Dec')] = pachis_delta_dec['50 Delta iVol - Dec'].astype(float)


/var/folders/zf/km2fq0xs06g8b5nj2t36d5fw0000gn/T/ipykernel_75578/307972202.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pachis_delta_dec['Date'] = pd.to_datetime(pachis_delta_dec['Date'])
/var/folders/zf/km2fq0xs06g8b5nj2t36d5fw0000gn/T/ipykernel_75578/307972202.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pachis_delta_dec.iloc[:, pachis_delta_dec.columns.get_loc(col)] = pachis_delta_dec[col].replace(' ', np.nan).astype(float)
/var/folders/zf/km2fq0xs06g8b5nj2t36d5fw0000gn/T/ipykernel_75578/307972202.py:

In [157]:
prompt_cols = ["Date.1", "50D.1", "25D Spread.1", "butterfly.1"]
pachis_delta_prompt = pachis_delta[prompt_cols]
pachis_delta_prompt.columns = ["Date", "50 Delta iVol - Prompt",
                               "25Δ Risk Reversal (Call - Put) - Prompt",
                               "Butterfly - Prompt"]
pachis_delta_prompt['Date'] = pd.to_datetime(pachis_delta_prompt['Date'])
pachis_delta_prompt = pachis_delta_prompt[pachis_delta_prompt['Date'].dt.year > 2018]
# First convert any string values to numeric, handling spaces
pachis_delta_prompt['50 Delta iVol - Prompt'] = pd.to_numeric(pachis_delta_prompt['50 Delta iVol - Prompt'].replace(' ', np.nan), errors='coerce')

/var/folders/zf/km2fq0xs06g8b5nj2t36d5fw0000gn/T/ipykernel_75578/2935795338.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pachis_delta_prompt['Date'] = pd.to_datetime(pachis_delta_prompt['Date'])


In [161]:
option_ts = pd.read_excel('../data/data_sheet_latest.xlsx', sheet_name='Option Time series')

In [209]:
# Create prompt dataframe
cols = ['Date', 'Call_OI-Prompt', 'Put_OI-Prompt', 'Dec_OI', 'Call/Put', 'Option%']
option_ts_prompt = option_ts[cols].copy()
option_ts_prompt.columns = ['Date', 'Call_OI-Prompt', 'Put_OI-Prompt', 'Dec_OI', 'Call/Put-Prompt', 'Option%-Prompt']

# Create december dataframe
cols = ['Date', 'Call/Put.1', 'Option%.1']
option_ts_dec = option_ts[cols].copy()
option_ts_dec.columns = ['Date', 'Call/Put-Dec', 'Option%-Dec']

# Merge the two dataframes on Date
option_ts_combined = pd.merge(option_ts_prompt, option_ts_dec, on='Date', how='outer')
option_ts_combined = option_ts_combined.sort_values(by='Date', ascending=False)
option_ts_combined = option_ts_combined[:-74]

In [235]:
combined_df = eua_options.merge(pachis_delta_dec, on='Date', how='left')\
    .merge(pachis_delta_prompt, on='Date', how='left')\
    .merge(option_ts_combined, on='Date', how='left')

In [257]:
combined_df = combined_df[combined_df['Date'].dt.year>=2023][:-70]

In [259]:
combined_df

,Date,Aggregate Put Open Interest (R1),Aggregate Call Open Interest (R1),Aggregate Open Interest (L1),OPTION OI%,PUT/CALL OI,Hist Vol - 1Y,iVol/Hist Vol Spread - Dec,50 Delta iVol - Dec,25Δ Risk Reversal (Call - Put) - Dec,...,50 Delta iVol - Prompt,25Δ Risk Reversal (Call - Put) - Prompt,Butterfly - Prompt,Call_OI-Prompt,Put_OI-Prompt,Dec_OI,Call/Put-Prompt,Option%-Prompt,Call/Put-Dec,Option%-Dec
0,2024-10-31,206996.0,321757.0,763691.0,0.409111,1.554412,28.706,8.366,37.072,-3.106,...,37.072,-3.106,1.1600,143541.0,111708.0,381074.0,1.284966,0.401131,1.284966,0.401131
1,2024-10-30,204588.0,322482.0,759379.0,0.409709,1.576251,28.694,8.398,37.092,-2.899,...,37.092,-2.899,0.9245,143741.0,111708.0,377423.0,1.286757,0.403635,1.286757,0.403635
2,2024-10-29,204538.0,320432.0,760706.0,0.408322,1.566614,28.205,8.773,36.978,-2.460,...,36.978,-2.460,1.0570,144571.0,111993.0,379961.0,1.290893,0.403070,1.290893,0.403070
3,2024-10-28,204623.0,321162.0,785605.0,0.400937,1.569530,28.121,8.829,36.95,-2.710,...,36.950,-2.710,1.1120,144571.0,112093.0,383677.0,1.289742,0.400824,1.289742,0.400824
4,2024-10-25,204723.0,321187.0,786404.0,0.400750,1.568886,28.34,8.534,36.874,-2.563,...,36.874,-2.563,1.0375,142771.0,111895.0,385602.0,1.275937,0.397749,1.275937,0.397749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,2023-04-20,104186.0,108732.0,503017.0,0.297399,1.043634,35.089,8.63,43.719,-3.627,...,41.104,-3.992,1.4260,18136.0,42833.0,259099.0,0.423412,0.190488,0.871061,0.297488
425,2023-04-19,102023.0,108612.0,502064.0,0.295546,1.064583,36.672,6.921,43.593,-3.359,...,40.937,-2.339,1.0805,16913.0,43758.0,262545.0,0.386512,0.187710,0.871061,0.294734
426,2023-04-18,102948.0,107389.0,502993.0,0.294866,1.043138,37.102,6.516,43.618,-2.995,...,41.498,-2.498,1.1590,16286.0,43460.0,264073.0,0.374735,0.184504,0.857175,0.291634
427,2023-04-17,102550.0,104912.0,502072.0,0.292392,1.023033,37.981,5.7,43.681,-3.051,...,42.012,-3.113,1.0855,16286.0,43360.0,265414.0,0.375600,0.183492,0.857418,0.290968


In [260]:
from IPython.display import display
import pandas as pd
import numpy as np
import streamlit as st
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from pathlib import Path
from pymongo import MongoClient
from datetime import datetime

from utils.dataset import MarketData, DataPreprocessor
from utils.data_processing import prepare_data

In [263]:
def load_and_preprocess_data():
    cot_df, auction_df, options_df, ta_df, fundamentals_df = MarketData.latest(Path('../data'))
    cot_df = cot_df.set_index('Date').resample('W', origin='end').mean().reset_index()
    auction_df = auction_df.set_index('Date').resample('D').mean().reset_index()

    auction_df = auction_df[7:]
    # auction_df.loc[:, 'Premium/discount-settle'] = auction_df['Premium/discount-settle'].ffill()
    auc_cols = ['Auc Price', 'Median Price', 'Cover Ratio', 'Spot Value', 
                'Auction Spot Diff', 'Median Spot Diff', 'Premium/discount-settle']
    auction_df.loc[:, auc_cols] = auction_df[auc_cols].ffill()

    merged_df = DataPreprocessor.engineer_auction_features(auction_df)
    return merged_df, options_df

In [266]:
merged_df, options_df = load_and_preprocess_data()

/Users/zeerakwyne/Documents/freelance/carbon-market-forecasting/notebooks/../utils/dataset.py:106: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pachis_delta_dec.iloc[:, pachis_delta_dec.columns.get_loc(col)] = pachis_delta_dec[col].replace(' ', np.nan).astype(float)
/Users/zeerakwyne/Documents/freelance/carbon-market-forecasting/notebooks/../utils/dataset.py:106: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pachis_delta_dec.iloc[:, pachis_delta_dec.columns.get_loc(col)] = pachis_delta_dec[col].replace(' ', np.nan).astype(float)


In [265]:

options_df

,Date,Aggregate Put Open Interest (R1),Aggregate Call Open Interest (R1),Aggregate Open Interest (L1),OPTION OI%,PUT/CALL OI,Hist Vol - 1Y,iVol/Hist Vol Spread - Dec,50 Delta iVol - Dec,25Δ Risk Reversal (Call - Put) - Dec,...,50 Delta iVol - Prompt,25Δ Risk Reversal (Call - Put) - Prompt,Butterfly - Prompt,Call_OI-Prompt,Put_OI-Prompt,Dec_OI,Call/Put-Prompt,Option%-Prompt,Call/Put-Dec,Option%-Dec
0,2024-10-31,206996.0,321757.0,763691.0,0.409111,1.554412,28.706,8.366,37.072,-3.106,...,37.072,-3.106,1.1600,143541.0,111708.0,381074.0,1.284966,0.401131,1.284966,0.401131
1,2024-10-30,204588.0,322482.0,759379.0,0.409709,1.576251,28.694,8.398,37.092,-2.899,...,37.092,-2.899,0.9245,143741.0,111708.0,377423.0,1.286757,0.403635,1.286757,0.403635
2,2024-10-29,204538.0,320432.0,760706.0,0.408322,1.566614,28.205,8.773,36.978,-2.460,...,36.978,-2.460,1.0570,144571.0,111993.0,379961.0,1.290893,0.403070,1.290893,0.403070
3,2024-10-28,204623.0,321162.0,785605.0,0.400937,1.569530,28.121,8.829,36.95,-2.710,...,36.950,-2.710,1.1120,144571.0,112093.0,383677.0,1.289742,0.400824,1.289742,0.400824
4,2024-10-25,204723.0,321187.0,786404.0,0.400750,1.568886,28.34,8.534,36.874,-2.563,...,36.874,-2.563,1.0375,142771.0,111895.0,385602.0,1.275937,0.397749,1.275937,0.397749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,2023-04-20,104186.0,108732.0,503017.0,0.297399,1.043634,35.089,8.63,43.719,-3.627,...,41.104,-3.992,1.4260,18136.0,42833.0,259099.0,0.423412,0.190488,0.871061,0.297488
425,2023-04-19,102023.0,108612.0,502064.0,0.295546,1.064583,36.672,6.921,43.593,-3.359,...,40.937,-2.339,1.0805,16913.0,43758.0,262545.0,0.386512,0.187710,0.871061,0.294734
426,2023-04-18,102948.0,107389.0,502993.0,0.294866,1.043138,37.102,6.516,43.618,-2.995,...,41.498,-2.498,1.1590,16286.0,43460.0,264073.0,0.374735,0.184504,0.857175,0.291634
427,2023-04-17,102550.0,104912.0,502072.0,0.292392,1.023033,37.981,5.7,43.681,-3.051,...,42.012,-3.113,1.0855,16286.0,43360.0,265414.0,0.375600,0.183492,0.857418,0.290968


In [267]:
merged_df

,Date,Auc Price,Median Price,Cover Ratio,Spot Value,Auction Spot Diff,Median Spot Diff,Premium/discount-settle,DayOfWeek,Month,...,Auc_Spot_Sum,Auc_to_Median_Spot_Ratio,Auc_Price_Squared,Cover_Ratio_Squared,Auc_Price_DayOfWeek_Interaction,Auc_Price_Month_Interaction,Auc_Price_Lag_Interaction,Auc_Price_RSI_Interaction,Complex_Interaction_1,Complex_Interaction_2
7,2018-06-11,15.70,15.59,2.03,15.69,0.01,-0.10,0.000000,0,6,...,31.39,0.064185,246.4900,4.1209,0.00,94.20,236.7560,609.697378,2.031294,0.2233
8,2018-06-12,15.08,15.13,1.63,15.20,-0.12,-0.07,0.000000,1,6,...,30.28,0.065572,227.4064,2.6569,15.08,90.48,236.7560,609.697378,1.617132,-0.0815
9,2018-06-13,15.08,15.13,1.63,15.20,-0.12,-0.07,0.000000,2,6,...,30.28,0.065572,227.4064,2.6569,30.16,90.48,227.4064,609.697378,1.617132,-0.0815
10,2018-06-14,15.44,15.30,2.27,15.39,0.05,-0.09,0.000000,3,6,...,30.83,0.065572,238.3936,5.1529,46.32,92.64,232.8352,609.697378,2.277375,0.3178
11,2018-06-15,14.62,14.69,1.68,14.79,-0.17,-0.10,0.000000,4,6,...,29.41,0.067291,213.7444,2.8224,58.48,87.72,225.7328,609.697378,1.660690,-0.1176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2338,2024-10-28,64.38,64.43,1.43,64.58,-0.20,-0.15,-0.003097,0,10,...,128.96,0.015473,4144.7844,2.0449,0.00,643.80,4269.6816,3382.443279,1.425571,-0.0715
2339,2024-10-29,66.42,66.11,1.65,66.33,0.09,-0.22,0.001357,1,10,...,132.75,0.015147,4411.6164,2.7225,66.42,664.20,4276.1196,4005.938114,1.652239,0.5115
2340,2024-10-30,66.42,66.11,1.65,66.33,0.09,-0.22,0.001357,2,10,...,132.75,0.015147,4411.6164,2.7225,132.84,664.20,4411.6164,4005.938114,1.652239,0.5115
2341,2024-10-31,64.78,64.62,1.51,64.72,0.06,-0.10,0.000927,3,10,...,129.50,0.015489,4196.4484,2.2801,194.34,647.80,4302.6876,3389.434192,1.511400,0.2416


In [277]:
pd.set_option('future.no_silent_downcasting', True)

Date                                       0
Aggregate Put Open Interest  (R1)          0
Aggregate Call Open Interest  (R1)         0
Aggregate Open Interest  (L1)              0
OPTION OI%                                 0
PUT/CALL OI                                0
Hist Vol - 1Y                              0
iVol/Hist Vol Spread - Dec                 0
50 Delta iVol - Dec                        0
25Δ Risk Reversal (Call - Put) - Dec       0
Butterfly - Dec                            0
50 Delta iVol - Prompt                     0
25Δ Risk Reversal (Call - Put) - Prompt    0
Butterfly - Prompt                         0
Call_OI-Prompt                             0
Put_OI-Prompt                              0
Dec_OI                                     0
Call/Put-Prompt                            0
Option%-Prompt                             0
Call/Put-Dec                               0
Option%-Dec                                0
Auc Price                                  0
dtype: int